In [2]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.bar import Frequency
from pyalgotrade import plotter
import numpy as np
from pyalgotrade.barfeed import quandlfeed
import datetime
from pyalgotrade.technical import ma
filename = "bank_nifty1.csv"
import pytz
firstDate = datetime.datetime(2012,5,30,9,15,0,0,pytz.utc)
endDate = datetime.datetime(2016,12,9,15,29,0,0,pytz.utc)

In [3]:
def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

In [4]:
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed)
        self.__sma10 = ma.SMA(feed[instrument].getCloseDataSeries(), 10)
        self.__instrument = instrument
        self.__sma20 = ma.SMA(feed[instrument].getCloseDataSeries(), 20)
# main event to update trading strategy
def onBars(self, bars):                
    self.portfolio_values.append(self.getBroker().getEquity())        
    if self.__sma20[-1] is None:
        return
    bar = bars[self.__instrument]                
                
    if self.__sma10[-1] > self.__sma20[-1]:   self.__signal = 1 # buying signal            
    elif self.__sma10[-1] < self.__sma20[-1]: self.__signal =-1 # selling signal             
    shares = 1
        
    if(self.__position) is None and self.__sma20[-2] is not None:
            # go into long position            
        if self.__sma10[-1] > self.__sma20[-1] and self.__sma10[-2] <= self.__sma20[-2]:
            self.info("short SMA > long SMA. RAISE BUY SIGNAL")                        
                #shares = int(self.getBroker().getCash() * 0.9 / bar.getClose())
            self.__position = self.enterLong(self.__instrument,shares,False)                                                    
            self.alert_message='Long position'         
            self.buy_signals.append(self.getCurrentDateTime())                                                       
            #short position
        elif self.__sma10[-1] < self.__sma20[-1] and self.__sma10[-2] >= self.__sma20[-2]: 
                self.info("short SMA < long SMA. RAISE SELL SIGNAL")   
                self.__position = self.enterShort(self.__instrument,shares,False)                   
                self.alert_message='Short position'       
                self.sell_signals.append(self.getCurrentDateTime())         
    elif self.__lastPrice is not None and self.getBroker().getPositions() != {}:                     
        pos = self.getBroker().getPositions()[self.__instrument]                        
            # take profit when we obtain >= $20
        if( np.sign(pos)*(bar.getClose() - self.__lastPrice) >= self.MAXPROFIT):
            self.alert_message = 'TAKE PROFIT'
            self.__position.exitMarket()
            self.__lastPrice = None
            # cut loss when we lose more than $10
        elif (np.sign(pos)*(self.__lastPrice - bar.getClose())) >= self.STOPLOSS:
            self.alert_message = 'STOP LOSS'
            self.__position.exitMarket()
            self.__lastPrice = None
        elif pos*self.__signal < 0:
            self.alert_message = "Reverse signal. TAKE PROFIT"
            self.__position.exitMarket()
            self.__lastPrice = None
            if self.__signal < 0:
                self.sell_signals.append(self.getCurrentDateTime())
            else:
                self.buy_signals.append(self.getCurrentDateTime())
            self.__last_exit_time = self.getCurrentDateTime() 
            
            
            
feed = csvfeed.GenericBarFeed(1*Frequency.MINUTE,pytz.utc)
#feed = quandlfeed.Feed()
feed.setBarFilter(csvfeed.DateRangeFilter(firstDate,endDate))
feed.addBarsFromCSV('bank_nifty',filename)

#cash = 10 ** 3 # 1,000 USD
myStrategy = MyStrategy(feed,'bank_nifty') #short and long moving average
#plt = plotter.StrategyPlotter(myStrategy, True, False, True)            
myStrategy.run()

TypeError: Can't instantiate abstract class MyStrategy with abstract methods onBars

In [ ]:
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self,instrument,feed,cash,sma):
        super(MyStrategy, self).__init__(feed)
        self.__instrument = instrument
        self.MAXPROFIT = 20; self.STOPLOSS = 10  
        self.getBroker().setAllowNegativeCash(True)
        
        # using for trading signal        
        self.__position = None        
        self.__price = feed[self.__instrument].getCloseDataSeries()        
        self.__sma10 = ma.SMA(self.__price,sma[0],maxLen=100000)
        self.__sma20 = ma.SMA(self.__price,sma[1],maxLen=100000)
        self.__lastPrice = 0 #last price. Use for take profit and cutloss                
        self.__signal = 0 #1: buying, -1: selling, 0: no change
        self.__last_exit_time = None
        
    def onEnterOk(self, position):        
        execInfo = position.getEntryOrder().getExecutionInfo()        
        self.info("%s %d at VND %s" %(self.alert_message,execInfo.getQuantity(), 
                                      ut.accountingFormat(execInfo.getPrice())))
        self.__lastPrice = execInfo.getPrice()                   
        self.record_detail_transaction(position)
        
    def onEnterCanceled(self, position):
        self.__position = None
        
    def onExitOk(self, position):        
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("%s %d at %s\n=================================" 
                  %(self.alert_message, 
                    execInfo.getQuantity(),'{:11,.2f}'.format(execInfo.getPrice())))        
        self.__position = None        
        self.record_detail_transaction(position, False) # log detail for later analysis
        
        # run before onEnterOk and onExitOk        
    def onOrderUpdated(self,order):    
        pass 

In [1]:
import pandas as pd

In [6]:
data = pd.read_csv('bank_nifty1.csv')

In [7]:
data.head()

,Date Time,Open,High,Low,Close,Volume
0,2012-05-30 09:15:00,9599.9004,9599.9004,9561.7998,9566.0000,21375
1,2012-05-30 09:16:00,9565.0996,9565.0996,9555.1504,9564.4004,15475
2,2012-05-30 09:17:00,9564.3496,9570.0000,9563.6504,9567.0000,13850
3,2012-05-30 09:18:00,9568.0000,9569.0000,9561.6504,9563.2500,11250
4,2012-05-30 09:19:00,9566.5498,9568.0000,9556.0000,9558.3496,16050


In [8]:
len(data)

412117

In [16]:
data.loc[0].Open

9599.9004

In [20]:
max([2,3])

3

In [17]:
data1 = data

In [11]:
for i*5 in range len(data1):
        data1.open = data1.loc[i*5]
        data1.close = data1.loc[i*5+4]
        data1.low = max([lambda x: ])
        

Date Time    2012-05-30 09:25:00
Open                        9542
High                        9545
Low                      9532.35
Close                     9543.5
Volume                     10650
Name: 10, dtype: object

In [23]:
data1 = pd.read_csv('bank_nifty1.csv',index_col=0)

In [25]:
ohlc_dict = {'Open':'first', 'High':'max', 'Low':'min', 'Close': 'last'}
data1 = data1.resample('15Min', how=ohlc_dict).dropna(how='any') 

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [3]:
ohlc_dict = {
    'Open':'first',
    'High':'max',
    'Low':'min',
    'Close':'last',
    'Volume':'sum'
    }

In [4]:
import pandas as pd
df = pd.read_csv('bank_nifty1.csv')

OSError: File b'bank_nifty1.csv' does not exist

In [49]:
df.head()

,Date Time,Open,High,Low,Close,Volume
0,2012-05-30 09:15:00,9599.9004,9599.9004,9561.7998,9566.0000,21375
1,2012-05-30 09:16:00,9565.0996,9565.0996,9555.1504,9564.4004,15475
2,2012-05-30 09:17:00,9564.3496,9570.0000,9563.6504,9567.0000,13850
3,2012-05-30 09:18:00,9568.0000,9569.0000,9561.6504,9563.2500,11250
4,2012-05-30 09:19:00,9566.5498,9568.0000,9556.0000,9558.3496,16050


In [50]:
df['Date Time'] = pd.to_datetime(df['Date Time'])
df.set_index('Date Time', inplace=True)

In [52]:
df = df.resample('5Min', how=ohlc_dict).dropna(how='any') 
cols=['Open', 'High', 'Low', 'Close','Volume']  
df = df[cols]
df.head(100)

/home/aryann/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  """Entry point for launching an IPython kernel.


,Open,High,Low,Close,Volume
Date Time,,,,,
2012-05-30 09:15:00,9599.9004,9599.9004,9555.1504,9558.3496,78000.0
2012-05-30 09:20:00,9555.0000,9558.3496,9531.0000,9541.0000,53575.0
2012-05-30 09:25:00,9542.0000,9545.0000,9532.3496,9539.0498,41175.0
2012-05-30 09:30:00,9540.0000,9542.0000,9522.4004,9522.5000,45450.0
2012-05-30 09:35:00,9522.0000,9522.0000,9503.0996,9514.9502,65400.0
2012-05-30 09:40:00,9514.9502,9524.9004,9511.3496,9524.0000,27025.0
2012-05-30 09:45:00,9525.0000,9534.0000,9521.3496,9530.0000,21200.0
2012-05-30 09:50:00,9528.0000,9536.7002,9528.0000,9528.2998,14475.0
2012-05-30 09:55:00,9528.2998,9537.6504,9526.9004,9536.4502,15000.0
